In [1]:
import re
import nltk
import joblib
import pickle
import pandas as pd
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

stemmer = PorterStemmer()

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\disha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
def loading_tweets(file_name):
    df = pd.read_csv(file_name, header = None)
    df = df.rename({0 : 'id', 1 : 'company', 2 : 'sentiment', 3 : 'raw_tweet'}, axis = 1)
    df = df[df['sentiment'] != 'Irrelevant']
    df = df[df['sentiment'] != 'Neutral']
    df = df.dropna()
    return df['raw_tweet'].values, df


def preprocessing_tweets(tweets): 
    processed_tweets = []
    for tweet in tqdm(tweets):
        tweet = re.sub(r'[^a-zA-Z]',' ',tweet)    
        tweet = tweet.lower()
        tweet = [word for word in tweet.split(' ') if not word in stopwords.words('english')]
        tweet = [stemmer.stem(word) for word in tweet]
        tweet = [word for word in tweet if len(word) != 0]
        tweet = ' '.join(tweet)
        processed_tweets.append(tweet)
    return tweets

In [6]:
train_tweets, df_train  = loading_tweets('twitter_training.csv')
train_tweets            = preprocessing_tweets(train_tweets)

tfidf = TfidfVectorizer()
train_tweets = tfidf.fit_transform(train_tweets)

x_train = train_tweets.toarray()
y_train = pd.get_dummies(df_train['sentiment']).values[:,1:]

100%|██████████| 43013/43013 [02:23<00:00, 300.44it/s]


In [12]:
test_tweets, df_test  = loading_tweets('twitter_validation.csv')
test_tweets            = preprocessing_tweets(test_tweets)

test_tweets = tfidf.transform(test_tweets)

x_test = test_tweets.toarray()
y_test = pd.get_dummies(df_test['sentiment']).values[:,1:]

100%|██████████| 543/543 [00:01<00:00, 277.54it/s]


In [13]:
mdl = MultinomialNB()

mdl.fit(x_train,y_train)

C:\Users\disha\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [14]:
print('Training Accuracy :', round(accuracy_score(mdl.predict(x_train), y_train)*100,3))
print('Testing Accuracy  :', round(accuracy_score(mdl.predict(x_test), y_test)*100,3))

Training Accuracy : 91.607
Testing Accuracy  : 94.107


In [15]:
joblib.dump(tfidf, 'tfidf_model.joblib')    
pickle.dump(mdl, open('model.mdl','wb'))  